---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevin-crotty.com)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [1]:
# Date Range (input a year)
start_yr = 1978
stop_yr  = 2023

# PULL DATA

In [2]:
from pandas_datareader import DataReader as pdr

files = ["DGS" + x for x in ["1", "2", "3", "5", "10", "30"]]
yield_changes = 100 * pdr(files, "fred", start=1920).dropna()
yield_changes = yield_changes.resample("M").last().diff()
yield_changes.index = yield_changes.index.to_period('M').astype(str)
yield_changes = yield_changes.dropna()
yield_changes.columns = [int(x[3:]) for x in yield_changes.columns]

# subset for data range
start = str(start_yr) + "-01"
stop  = str(stop_yr)  + "-12"
df = yield_changes.loc[start:stop]

# CALCULATIONS

In [3]:
import numpy as np
import pandas as pd

# principal components
w, v = np.linalg.eigh(df.cov())

# amount of the variation in yields that each of the shocks explains
n = len(df.columns)
w = pd.Series(w[range(n-1, -1, -1)])
w = w.cumsum() / w.sum()
w.index = [f'first {i}' for i in range(1, n + 1)]
w = pd.DataFrame(w).reset_index()
w.columns = ['first n components', 'percent explained']

# how the yield at each of the maturities changes in response to a unit shock
v = pd.DataFrame(v, index=df.columns)
v = v[range(n-1, -1, -1)]
for i in v.columns:
    v[i] = v[i].iloc[0] * v[i]
v = (100*v).round(1)
v = v.reset_index()
v.columns = ['maturity'] + [i for i in range(1, n+1)]
v['maturity'] = [
    "1 year",
    "2 years",
    "3 years",
    "5 years",
    "10 years",
    "30 years"
]

# TABLES

In [4]:
w

,first n components,percent explained
0,first 1,0.916392
1,first 2,0.979817
2,first 3,0.994573
3,first 4,0.997414
4,first 5,0.999013
5,first 6,1.000000


In [5]:
v

,maturity,1,2,3,4,5,6
0,1 year,23.0,35.5,27.5,11.8,1.5,0.7
1,2 years,22.2,14.8,-10.2,-16.9,-6.1,-3.9
2,3 years,21.1,1.7,-21.8,-10.1,3.2,6.0
3,5 years,19.7,-14.4,-20.9,14.0,5.7,-4.2
4,10 years,16.3,-29.1,3.4,14.9,-7.7,2.2
5,30 years,13.3,-31.8,31.0,-15.2,3.6,-0.8


# FIGURE

In [6]:
import plotly.graph_objects as go
fig = go.Figure()
trace1 = go.Scatter(x=v.maturity, y=v[1],
                    mode='lines+markers',
                    name='first',
                    )
trace2 = go.Scatter(x=v.maturity, y=v[2],
                    mode='lines+markers',
                    name='second',
                    )
trace3 = go.Scatter(x=v.maturity, y=v[3],
                    mode='lines+markers',
                    name='third',
                    )
for trace in [trace1, trace2, trace3]:
    fig.add_trace(trace)
fig.update_layout(
    hovermode='x unified',
    xaxis_tickformat=',.0f',
    yaxis_tickformat=',.0f',
    xaxis_title="Maturity",
    yaxis_title='Yield Changes (basis points)',
    template="plotly_white",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    ) 
)
